In [7]:
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist
from keras.applications import DenseNet121

In [8]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

X_train = np.array([cv2.resize(im, (128, 128)) for im in X_train])
X_test = np.array([cv2.resize(im, (128, 128)) for im in X_test])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255.0
X_test /= 255.0

X_train = np.stack((X_train,)*3, axis=-1)
X_test = np.stack((X_test,)*3, axis=-1)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [11]:
import numpy as np
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Load DenseNet model with pre-trained weights from Keras
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
base_model.trainable = False

# Add new layers for Fashion MNIST classification
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(10, activation='softmax')(x)

# Create a new model
model = models.Model(inputs=base_model.input, outputs=x)

In [12]:
from keras.optimizers import Adam

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
batch_size = 32
epochs = 10

history = model.fit(X_train, y_train, batch_size=batch_size, epochs = epochs)

Epoch 1/10
1875/1875 [==============================] - 1295s 687ms/step - loss: 0.3967 - accuracy: 0.8596
Epoch 2/10
1875/1875 [==============================] - 1324s 706ms/step - loss: 0.2927 - accuracy: 0.8930
Epoch 3/10
1875/1875 [==============================] - 1321s 705ms/step - loss: 0.2642 - accuracy: 0.9014
Epoch 4/10
1875/1875 [==============================] - 1333s 711ms/step - loss: 0.2454 - accuracy: 0.9083
Epoch 5/10
1875/1875 [==============================] - 1340s 714ms/step - loss: 0.2276 - accuracy: 0.9147
Epoch 6/10
1875/1875 [==============================] - 1343s 716ms/step - loss: 0.2124 - accuracy: 0.9209
Epoch 7/10
1875/1875 [==============================] - 1318s 703ms/step - loss: 0.1988 - accuracy: 0.9243
Epoch 8/10
1875/1875 [==============================] - 1281s 683ms/step - loss: 0.1865 - accuracy: 0.9311
Epoch 9/10
1875/1875 [==============================] - 1272s 679ms/step - loss: 0.1764 - accuracy: 0.9343
Epoch 10/10
1875/1875 [==============

In [14]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

313/313 - 216s - loss: 0.3363 - accuracy: 0.8897 - 216s/epoch - 691ms/step

Test accuracy: 0.8896999955177307


In [16]:
model.save('NN/model_fashion_mnist_densenet.h5')